In [25]:
import numpy as np

def is_pos_def(x):
    counter = 0
    for i in np.linalg.eigvals(x):
        #print(i)
        if i < -0.0001:
            counter += 1
    if counter == 0:
        return True
    else:
        return False

In [26]:
import pandas as pd
from dtaidistance import dtw

dataset = pd.read_csv('coffee.txt', sep = '  ', engine='python', header = None)
dataset = dataset.iloc[:, :-1].values
data_train = dataset[:, 1:101]
target_train = dataset[:, 0]
l = len(data_train)
D_train = np.zeros((l, l))
for i in range(l):
    for j in range(l):
        D_train[i][j] = dtw.distance(data_train[i], data_train[j])
print(D_train)


[[0.         0.57429662 0.42516589 ... 0.39008773 0.66528083 0.6065221 ]
 [0.57429662 0.         0.37104667 ... 0.37254401 0.30006547 0.34707764]
 [0.42516589 0.37104667 0.         ... 0.32906845 0.48273798 0.35006481]
 ...
 [0.39008773 0.37254401 0.32906845 ... 0.         0.46941068 0.42974104]
 [0.66528083 0.30006547 0.48273798 ... 0.46941068 0.         0.44756658]
 [0.6065221  0.34707764 0.35006481 ... 0.42974104 0.44756658 0.        ]]


In [27]:
is_pos_def(D_train)

False

In [28]:
import math
A_train = np.zeros((l, l))
for i in range(l):
    for j in range(l):
        A_train[i][j] = math.exp(-D_train[i][j])

print(A_train)

[[1.         0.5631008  0.65366134 ... 0.67699748 0.51412913 0.54524388]
 [0.5631008  1.         0.69001174 ... 0.68897933 0.74076972 0.70675045]
 [0.65366134 0.69001174 1.         ... 0.71959376 0.61709149 0.70464242]
 ...
 [0.67699748 0.68897933 0.71959376 ... 1.         0.6253707  0.65067757]
 [0.51412913 0.74076972 0.61709149 ... 0.6253707  1.         0.63918166]
 [0.54524388 0.70675045 0.70464242 ... 0.65067757 0.63918166 1.        ]]


In [29]:
is_pos_def(A_train)

True

In [30]:
import numpy as np
import matplotlib.pyplot as plt

from functools import partial
from sklearn import svm
from sklearn.model_selection import train_test_split

# 200 x 2d timeseries of length 100
#X = np.random.randn(200, 100, 2)
#y = np.random.randint(0, 2, 200)


X_train, X_test, y_train, y_test = train_test_split(data_train, target_train, train_size=0.8)

x_shape = X_train.shape


In [31]:
def kernel_matrix(X1, X2, f):
    D = [[f(X1[i], X2[j]) for j in range(len(X2))]for i in range(len(X1))]
    return np.array(D)

#kernel_matrix(X_test, X_test, lambda x, y: ((x - y)**2).sum())

In [32]:
clf = svm.SVC(kernel="precomputed")
clf.fit(kernel_matrix(X_train, X_train, lambda x, y: ((x - y)**2).sum()), y_train)

print(f"Train score with l2: {clf.score(kernel_matrix(X_train, X_train, lambda x, y: ((x - y)**2).sum()), y_train):.3f}")
print(f"Test score with l2: {clf.score(kernel_matrix(X_test, X_train, lambda x, y: ((x - y)**2).sum()), y_test):.3f}")

Train score with l2: 0.341
Test score with l2: 0.250


In [33]:
clf = svm.SVC(kernel="precomputed")
D = [[dtw.distance(X_train[i], X_train[j]) for j in range (len(X_train))]for i in range(len(X_train))]
C = [[dtw.distance(X_test[i], X_train[j]) for j in range (len(X_train))]for i in range(len(X_test))]
clf.fit(np.array(D), y_train)

print(f"Train score with DTW: {clf.score(np.array(D), y_train):.3f}")
print(f"Test score with DTW: {clf.score(np.array(C), y_test):.3f}")

Train score with DTW: 0.341
Test score with DTW: 0.250


In [34]:
A = np.zeros((len(X_train), len(X_train)))
for i in range(len(X_train)):
    for j in range(len(X_train)):
        A[i][j] = math.exp(-D[i][j])

clf = svm.SVC(kernel="precomputed")

clf.fit(np.array(A), y_train)

print(f"Train score with DTW (upgrade): {clf.score(np.array(A), y_train):.3f}")
print(f"Test score with DTW: {clf.score(np.array(C), y_test):.3f}")


Train score with DTW (upgrade): 0.864
Test score with DTW: 0.250
